# DC Child Tax Credit Comprehensive Reform Analysis

This notebook analyzes DC CTC reforms with various parameter combinations:
- Age thresholds: Under 6, 8, 12, or 18
- Amount per child: $420, $1,000, $1,500
- Child limit: 3 or unlimited
- Phase-out amount: $20, $50, $100 (reduction per $1,000 over threshold)
- Lower phase-out thresholds: $50k (HoH/Single/Surviving), $75k (Joint), $37.5k (Separate)

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
import itertools

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load DC dataset
sim = Microsimulation(dataset='hf://policyengine/test/DC.h5')

## Baseline: Zero Out Current DC CTC

Current DC CTC provides $420 per child under age 6, with income phase-outs starting at $160k (single)/$240k (joint).

In [3]:
def create_baseline_reform():
    """Zero out the existing DC CTC"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

baseline_reform = create_baseline_reform()
baseline_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=baseline_reform)

In [4]:
# Calculate baseline cost
current_ctc = sim.calculate("dc_ctc", period=2025, map_to="household")
baseline_ctc = baseline_sim.calculate("dc_ctc", period=2025, map_to="household")

current_cost = current_ctc.sum()
print(f"Current DC CTC annual cost: ${current_cost:,.0f}")
print(f"Baseline DC CTC cost (zeroed out): ${baseline_ctc.sum():,.0f}")

Current DC CTC annual cost: $17,971,915
Baseline DC CTC cost (zeroed out): $0


## Reform Scenarios

Testing all combinations of reform parameters.

In [5]:
def create_reform(age_threshold, amount_per_child, child_cap, phase_out_amount):
    """
    Create a DC CTC reform with specified parameters.
    
    Parameters:
    - age_threshold: Maximum age for eligible children (6, 8, 12, or 18)
    - amount_per_child: Dollar amount per child (420, 1000, or 1500)
    - child_cap: Maximum number of children (3 or 999 for unlimited)
    - phase_out_amount: Reduction per $1,000 over income threshold (20, 50, or 100)
    
    All reforms use lower income thresholds:
    - Single/HoH/Surviving Spouse: $50,000
    - Joint: $75,000
    - Separate: $37,500
    """
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": age_threshold
            },
            "gov.states.dc.tax.income.credits.ctc.child.child_cap": {
                "2025-01-01.2100-12-31": child_cap
            },
            "gov.states.dc.tax.income.credits.ctc.phase_out.amount": {
                "2025-01-01.2100-12-31": phase_out_amount
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2025-01-01.2100-12-31": 50_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2025-01-01.2100-12-31": 75_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2025-01-01.2100-12-31": 37_500
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 50_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 50_000
            }
        },
        country_id="us",
    )

In [6]:
# Define parameter combinations
age_thresholds = [6, 8, 12, 18]
amounts = [420, 1000, 1500]
child_caps = [3, 999]  # 999 represents unlimited
phase_out_amounts = [20, 50, 100]

# Generate all combinations
combinations = list(itertools.product(age_thresholds, amounts, child_caps, phase_out_amounts))
print(f"Total number of reform scenarios: {len(combinations)}")

Total number of reform scenarios: 72


In [7]:
# Run all reforms and collect results
results = []

for i, (age, amount, cap, phase_out) in enumerate(combinations, 1):
    print(f"Running reform {i}/{len(combinations)}: Age<{age}, Amount=${amount}, Cap={cap if cap < 999 else 'None'}, PhaseOut=${phase_out}")
    
    reform = create_reform(age, amount, cap, phase_out)
    reform_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=reform)
    reform_ctc = reform_sim.calculate("dc_ctc", period=2025, map_to="household")
    total_cost = reform_ctc.sum()
    
    results.append({
        "Age Threshold": f"Under {age}",
        "Amount per Child": f"${amount:,}",
        "Child Limit": str(cap) if cap < 999 else "None",
        "Phase-out Amount": f"${phase_out}",
        "Annual Cost": f"${total_cost:,.0f}",
        "Cost (Numeric)": total_cost,
        "Age (Numeric)": age,
        "Amount (Numeric)": amount,
        "Cap (Numeric)": cap,
        "Phase-out (Numeric)": phase_out
    })

print("\nAll reforms completed!")

Running reform 1/72: Age<6, Amount=$420, Cap=3, PhaseOut=$20
Running reform 2/72: Age<6, Amount=$420, Cap=3, PhaseOut=$50
Running reform 3/72: Age<6, Amount=$420, Cap=3, PhaseOut=$100
Running reform 4/72: Age<6, Amount=$420, Cap=None, PhaseOut=$20
Running reform 5/72: Age<6, Amount=$420, Cap=None, PhaseOut=$50
Running reform 6/72: Age<6, Amount=$420, Cap=None, PhaseOut=$100
Running reform 7/72: Age<6, Amount=$1000, Cap=3, PhaseOut=$20
Running reform 8/72: Age<6, Amount=$1000, Cap=3, PhaseOut=$50
Running reform 9/72: Age<6, Amount=$1000, Cap=3, PhaseOut=$100
Running reform 10/72: Age<6, Amount=$1000, Cap=None, PhaseOut=$20
Running reform 11/72: Age<6, Amount=$1000, Cap=None, PhaseOut=$50
Running reform 12/72: Age<6, Amount=$1000, Cap=None, PhaseOut=$100
Running reform 13/72: Age<6, Amount=$1500, Cap=3, PhaseOut=$20
Running reform 14/72: Age<6, Amount=$1500, Cap=3, PhaseOut=$50
Running reform 15/72: Age<6, Amount=$1500, Cap=3, PhaseOut=$100
Running reform 16/72: Age<6, Amount=$1500, Cap=

In [8]:
# Create DataFrame and display results
df = pd.DataFrame(results)

# Display table
print("\n" + "="*120)
print("DC CHILD TAX CREDIT REFORM SCENARIOS")
print("Lower Income Thresholds: $50k (Single/HoH/Surviving), $75k (Joint), $37.5k (Separate)")
print("="*120)
print(df[['Age Threshold', 'Amount per Child', 'Child Limit', 'Phase-out Amount', 'Annual Cost']].to_string(index=False))
print("="*120)


DC CHILD TAX CREDIT REFORM SCENARIOS
Lower Income Thresholds: $50k (Single/HoH/Surviving), $75k (Joint), $37.5k (Separate)
Age Threshold Amount per Child Child Limit Phase-out Amount  Annual Cost
      Under 6             $420           3              $20  $14,320,670
      Under 6             $420           3              $50  $13,841,326
      Under 6             $420           3             $100  $13,615,555
      Under 6             $420        None              $20  $14,555,760
      Under 6             $420        None              $50  $14,076,416
      Under 6             $420        None             $100  $13,850,645
      Under 6           $1,000           3              $20  $36,101,632
      Under 6           $1,000           3              $50  $34,013,107
      Under 6           $1,000           3             $100  $33,109,273
      Under 6           $1,000        None              $20  $36,661,372
      Under 6           $1,000        None              $50  $34,572,846


In [9]:
# Export to CSV
csv_filename = "dc_ctc_reforms_comprehensive.csv"
df[['Age Threshold', 'Amount per Child', 'Child Limit', 'Phase-out Amount', 'Annual Cost']].to_csv(csv_filename, index=False)
print(f"\nResults exported to {csv_filename}")


Results exported to dc_ctc_reforms_comprehensive.csv


In [10]:
# Summary statistics
print("\nSUMMARY STATISTICS")
print(f"Minimum cost: ${df['Cost (Numeric)'].min():,.0f}")
print(f"Maximum cost: ${df['Cost (Numeric)'].max():,.0f}")
print(f"Mean cost: ${df['Cost (Numeric)'].mean():,.0f}")
print(f"Median cost: ${df['Cost (Numeric)'].median():,.0f}")


SUMMARY STATISTICS
Minimum cost: $13,615,555
Maximum cost: $167,183,711
Mean cost: $59,920,569
Median cost: $49,366,021


In [11]:
# Show reforms grouped by age threshold
print("\nRESULTS BY AGE THRESHOLD")
for age in age_thresholds:
    subset = df[df['Age (Numeric)'] == age]
    print(f"\nUnder {age}:")
    print(subset[['Amount per Child', 'Child Limit', 'Phase-out Amount', 'Annual Cost']].to_string(index=False))


RESULTS BY AGE THRESHOLD

Under 6:
Amount per Child Child Limit Phase-out Amount Annual Cost
            $420           3              $20 $14,320,670
            $420           3              $50 $13,841,326
            $420           3             $100 $13,615,555
            $420        None              $20 $14,555,760
            $420        None              $50 $14,076,416
            $420        None             $100 $13,850,645
          $1,000           3              $20 $36,101,632
          $1,000           3              $50 $34,013,107
          $1,000           3             $100 $33,109,273
          $1,000        None              $20 $36,661,372
          $1,000        None              $50 $34,572,846
          $1,000        None             $100 $33,669,012
          $1,500           3              $20 $56,226,654
          $1,500           3              $50 $52,092,604
          $1,500           3             $100 $50,344,382
          $1,500        None        

In [12]:
# Show reforms grouped by amount
print("\nRESULTS BY AMOUNT PER CHILD")
for amount in amounts:
    subset = df[df['Amount (Numeric)'] == amount]
    print(f"\n${amount:,} per child:")
    print(subset[['Age Threshold', 'Child Limit', 'Phase-out Amount', 'Annual Cost']].to_string(index=False))


RESULTS BY AMOUNT PER CHILD

$420 per child:
Age Threshold Child Limit Phase-out Amount Annual Cost
      Under 6           3              $20 $14,320,670
      Under 6           3              $50 $13,841,326
      Under 6           3             $100 $13,615,555
      Under 6        None              $20 $14,555,760
      Under 6        None              $50 $14,076,416
      Under 6        None             $100 $13,850,645
      Under 8           3              $20 $18,751,082
      Under 8           3              $50 $17,958,771
      Under 8           3             $100 $17,637,315
      Under 8        None              $20 $19,107,110
      Under 8        None              $50 $18,314,799
      Under 8        None             $100 $17,993,343
     Under 12           3              $20 $26,912,518
     Under 12           3              $50 $25,450,678
     Under 12           3             $100 $24,910,030
     Under 12        None              $20 $28,476,680
     Under 12      

In [13]:
# Compare effect of child cap
print("\nIMPACT OF CHILD CAP (3 vs Unlimited) by Age and Amount")
print("Using Phase-out = $20\n")
for age in age_thresholds:
    for amount in amounts:
        capped_data = df[(df['Age (Numeric)'] == age) & (df['Amount (Numeric)'] == amount) & (df['Cap (Numeric)'] == 3) & (df['Phase-out (Numeric)'] == 20)]
        unlimited_data = df[(df['Age (Numeric)'] == age) & (df['Amount (Numeric)'] == amount) & (df['Cap (Numeric)'] == 999) & (df['Phase-out (Numeric)'] == 20)]
        
        if len(capped_data) > 0 and len(unlimited_data) > 0:
            capped = capped_data['Cost (Numeric)'].values[0]
            unlimited = unlimited_data['Cost (Numeric)'].values[0]
            difference = unlimited - capped
            pct_increase = (difference / capped) * 100 if capped > 0 else 0
            print(f"Age<{age}, ${amount:,}/child: Cap 3=${capped:,.0f}, Unlimited=${unlimited:,.0f}, Diff=${difference:,.0f} (+{pct_increase:.1f}%)")


IMPACT OF CHILD CAP (3 vs Unlimited) by Age and Amount
Using Phase-out = $20

Age<6, $420/child: Cap 3=$14,320,670, Unlimited=$14,555,760, Diff=$235,090 (+1.6%)
Age<6, $1,000/child: Cap 3=$36,101,632, Unlimited=$36,661,372, Diff=$559,739 (+1.6%)
Age<6, $1,500/child: Cap 3=$56,226,654, Unlimited=$57,066,263, Diff=$839,609 (+1.5%)
Age<8, $420/child: Cap 3=$18,751,082, Unlimited=$19,107,110, Diff=$356,028 (+1.9%)
Age<8, $1,000/child: Cap 3=$47,539,975, Unlimited=$48,387,661, Diff=$847,686 (+1.8%)
Age<8, $1,500/child: Cap 3=$74,144,075, Unlimited=$75,415,604, Diff=$1,271,529 (+1.7%)
Age<12, $420/child: Cap 3=$26,912,518, Unlimited=$28,476,680, Diff=$1,564,162 (+5.8%)
Age<12, $1,000/child: Cap 3=$68,648,053, Unlimited=$72,372,248, Diff=$3,724,195 (+5.4%)
Age<12, $1,500/child: Cap 3=$106,877,819, Unlimited=$112,464,111, Diff=$5,586,292 (+5.2%)
Age<18, $420/child: Cap 3=$39,362,370, Unlimited=$42,068,611, Diff=$2,706,241 (+6.9%)
Age<18, $1,000/child: Cap 3=$101,055,690, Unlimited=$107,513,60

In [14]:
# Compare effect of phase-out amount
print("\nIMPACT OF PHASE-OUT AMOUNT ($20 vs $50 vs $100)")
print("Using Child Cap = None\n")
for age in age_thresholds:
    for amount in amounts:
        phase20_data = df[(df['Age (Numeric)'] == age) & (df['Amount (Numeric)'] == amount) & (df['Cap (Numeric)'] == 999) & (df['Phase-out (Numeric)'] == 20)]
        phase50_data = df[(df['Age (Numeric)'] == age) & (df['Amount (Numeric)'] == amount) & (df['Cap (Numeric)'] == 999) & (df['Phase-out (Numeric)'] == 50)]
        phase100_data = df[(df['Age (Numeric)'] == age) & (df['Amount (Numeric)'] == amount) & (df['Cap (Numeric)'] == 999) & (df['Phase-out (Numeric)'] == 100)]
        
        if len(phase20_data) > 0 and len(phase50_data) > 0 and len(phase100_data) > 0:
            cost20 = phase20_data['Cost (Numeric)'].values[0]
            cost50 = phase50_data['Cost (Numeric)'].values[0]
            cost100 = phase100_data['Cost (Numeric)'].values[0]
            diff_50 = cost20 - cost50
            diff_100 = cost20 - cost100
            print(f"Age<{age}, ${amount:,}/child: $20=${cost20:,.0f}, $50=${cost50:,.0f} (-${diff_50:,.0f}), $100=${cost100:,.0f} (-${diff_100:,.0f})")


IMPACT OF PHASE-OUT AMOUNT ($20 vs $50 vs $100)
Using Child Cap = None

Age<6, $420/child: $20=$14,555,760, $50=$14,076,416 (-$479,344), $100=$13,850,645 (-$705,115)
Age<6, $1,000/child: $20=$36,661,372, $50=$34,572,846 (-$2,088,525), $100=$33,669,012 (-$2,992,360)
Age<6, $1,500/child: $20=$57,066,263, $50=$52,932,212 (-$4,134,051), $100=$51,183,991 (-$5,882,273)
Age<8, $420/child: $20=$19,107,110, $50=$18,314,799 (-$792,311), $100=$17,993,343 (-$1,113,767)
Age<8, $1,000/child: $20=$48,387,661, $50=$45,361,928 (-$3,025,733), $100=$43,918,257 (-$4,469,404)
Age<8, $1,500/child: $20=$75,415,604, $50=$69,783,890 (-$5,631,714), $100=$67,037,252 (-$8,378,352)
Age<12, $420/child: $20=$28,476,680, $50=$26,901,509 (-$1,575,171), $100=$26,313,513 (-$2,163,167)
Age<12, $1,000/child: $20=$72,372,248, $50=$67,576,840 (-$4,795,408), $100=$64,627,294 (-$7,744,954)
Age<12, $1,500/child: $20=$112,464,111, $50=$104,262,531 (-$8,201,580), $100=$99,438,296 (-$13,025,815)
Age<18, $420/child: $20=$42,068,6